<a href="https://colab.research.google.com/github/maheshmistry/ProgrammT/blob/main/Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib
!pip install faker

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from faker import Faker
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random



dbo = Namespace("http://dbpedia.org/ontology/")
TOnto = Namespace("http://visualdataweb.org/MCompanyTest/")

In [ ]:
fake = Faker()
g = Graph()

g.bind("TOnto", 'http://visualdataweb.org/MCompanyTest/')
g.bind("dbo", 'http://dbpedia.org/ontology/')

g.parse("/content/drive/My Drive/ColabNotebooks/NewOntology.rdf")

file:///content/drive/My Drive/ColabNotebooks/NewOntology.rdf does not look like a valid URI, trying to serialize this will break.


FileNotFoundError: ignored

In [ ]:
qres = g.query(
    """SELECT ?s ?p ?o
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s ?p ?o.
           filter (datatype(?o) != '')
       }""")

for row in qres:
    print("%s -- %s -- %s" % row)

In [ ]:
enumb = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s dbo:numberOfEmployees ?o.
       }""")

avgSal = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s TOnto:avgSalary ?o.
       }""")
avgGenRat = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE{
          ?s rdf:type TOnto:Company.
          ?s TOnto:genderRatio ?o.
       }"""
)

for row in enumb:
  emp = row.avg
for row in avgSal:
  avgSale = row.avg
for row in avgGenRat:
  avgGenRatio = row.avg

emp = int(emp)
avgSale = float(avgSale)


In [ ]:
synthGender = np.random.choice(["Male", "Female"], emp, p=[(avgGenRatio.value/100), ((100-avgGenRatio.value)/100)])

emp_roles = ['Engineer','Senior Engineer','Manager']
company = pd.DataFrame([
          {
              "FirstName": fake.first_name_male() if synthGender[n]=='Male' else fake.first_name_female(),
              "LastName": fake.last_name(),
              "Gender" : synthGender[n],
              "Salary" : random.randint(int(avgSale - (avgSale * .20)), int(avgSale + (avgSale * .20))),
              "Roles" : "CEO" if n == 0 else np.random.choice(emp_roles, p=[0.6, 0.3, 0.1])
          }for n in range(emp)
])


In [ ]:
genDist = (company['Gender'].value_counts(normalize=True) * 100)

print("New Company stats:"
      "\n\nNo. of Employees:" , emp ,
      "\n\nGender Ratio(Male Percent):\n", genDist,
      "\n\nAverage Salary:", sum(company["Salary"])/emp,
      "\n\nRole Distribution:\n", company["Roles"].value_counts())